In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import FunctionTransformer, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.model_selection import TimeSeriesSplit

In [2]:
df = pd.read_csv('lic_clean_49_24.csv')

In [3]:
#2021, année covid // interpolate
df.head()
df = df.sort_values("annee")

df=df.drop(df[df['annee']==2021].index)
df=df.drop(df[df['annee']==2024].index)
df['annee'] = df['annee'].apply(int)

#2016 : changement méthodo, environ +2,6
mask = (df["annee"] >= 2016) & (df["annee"] <= 2023)
df.loc[mask, "total"] = df.loc[mask, "total"] / 2.6

df.head()

,Unnamed: 0,annee,olympique,non-olympique,affinitaire,scolaire,total
0,0,1949,949000.0,322000.0,478000.0,187000.0,1935000.0
1,1,1950,1001000.0,312000.0,510000.0,331000.0,2154000.0
2,2,1951,1220000.0,324000.0,407000.0,338000.0,2289000.0
3,3,1952,1226000.0,347000.0,406000.0,382000.0,2360000.0
4,4,1953,1057000.0,372000.0,425000.0,399000.0,2252000.0


In [4]:
df = df.sort_values("annee")
X = df[["annee"]]
y = df[["total"]]

split_index = int(len(df) * 0.6)

X_train = X.iloc[:split_index]
X_test  = X.iloc[split_index:]

y_train = y.iloc[:split_index]
y_test  = y.iloc[split_index:]

print("Années dans X_train :", X_train["annee"].unique())
print("Années dans X_test :", X_test["annee"].unique())

Années dans X_train : [1949 1950 1951 1952 1953 1954 1955 1956 1957 1958 1959 1960 1961 1962
 1963 1964 1965 1966 1967 1968 1969 1970 1971 1972 1973 1974 1975 1976
 1977 1978 1979 1980 1981 1982 1983 1984 1985 1986 1987 1988 1989 1990
 1991 1992]
Années dans X_test : [1993 1994 1995 1996 1997 1998 1999 2000 2001 2002 2003 2004 2005 2006
 2007 2008 2009 2010 2011 2012 2013 2014 2015 2016 2017 2018 2019 2020
 2022 2023]


In [5]:
X_train.info()

<class 'pandas.DataFrame'>
Index: 44 entries, 0 to 43
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   annee   44 non-null     int64
dtypes: int64(1)
memory usage: 704.0 bytes


In [6]:
X.shape

(74, 1)

In [7]:
# date_features=["annee"]

# def dates_transfo(X):
#     print(X.shape)
#     X = pd.DataFrame(X, columns=["annee"])
#     X["annee"] = pd.to_datetime(X["annee"])
    
#     return pd.DataFrame({
#         "annee":  X["annee"].dt.year})

# date_transformer = FunctionTransformer(dates_transfo, validate=False)

# feature_encoder = ColumnTransformer(
#     transformers=[
#        ('date', date_transformer, date_features),
#     ]
# )

# categorical_features=["annee"]

# categorical_transformer = OneHotEncoder(handle_unknown='ignore')

# feature_encoder = ColumnTransformer(
#     transformers=[
#        ('cat', categorical_transformer, categorical_features),
#     ]
# )


# X_train = feature_encoder.fit_transform(X_train)
# X_test = feature_encoder.transform(X_test)

In [8]:
regressor=LinearRegression()
regressor.fit(X_train, y_train)

,"fit_intercept fit_intercept: bool, default=TrueWhether to calculate the intercept for this model. If setto False, no intercept will be used in calculations(i.e. data is expected to be centered).",True
,"copy_X copy_X: bool, default=TrueIf True, X will be copied; else, it may be overwritten.",True
,"tol tol: float, default=1e-6The precision of the solution (`coef_`) is determined by `tol` whichspecifies a different convergence criterion for the `lsqr` solver.`tol` is set as `atol` and `btol` of :func:`scipy.sparse.linalg.lsqr` whenfitting on sparse training data. This parameter has no effect when fittingon dense data... versionadded:: 1.7",1e-06
,"n_jobs n_jobs: int, default=NoneThe number of jobs to use for the computation. This will only providespeedup in case of sufficiently large problems, that is if firstly`n_targets > 1` and secondly `X` is sparse or if `positive` is setto `True`. ``None`` means 1 unless in a:obj:`joblib.parallel_backend` context. ``-1`` means using allprocessors. See :term:`Glossary ` for more details.",None
,"positive positive: bool, default=FalseWhen set to ``True``, forces the coefficients to be positive. Thisoption is only supported for dense arrays.For a comparison between a linear regression model with positive constraintson the regression coefficients and a linear regression without such constraints,see :ref:`sphx_glr_auto_examples_linear_model_plot_nnls.py`... versionadded:: 0.24",False


In [9]:
print(df['annee'].value_counts())

annee
1949    1
1950    1
1951    1
1952    1
1953    1
       ..
2018    1
2019    1
2020    1
2022    1
2023    1
Name: count, Length: 74, dtype: int64


In [10]:
print(X_train.shape)
print(X_train[:5])
print(y_train[:5])

(44, 1)
   annee
0   1949
1   1950
2   1951
3   1952
4   1953
       total
0  1935000.0
1  2154000.0
2  2289000.0
3  2360000.0
4  2252000.0


In [11]:
Y_pred= regressor.predict(X_test)
print(Y_pred)

[[13679206.13107824]
 [13988560.74700487]
 [14297915.36293161]
 [14607269.97885835]
 [14916624.59478498]
 [15225979.21071172]
 [15535333.82663846]
 [15844688.4425652 ]
 [16154043.05849183]
 [16463397.67441857]
 [16772752.29034531]
 [17082106.90627193]
 [17391461.52219868]
 [17700816.13812542]
 [18010170.75405216]
 [18319525.36997879]
 [18628879.98590553]
 [18938234.60183227]
 [19247589.21775889]
 [19556943.83368564]
 [19866298.44961238]
 [20175653.06553912]
 [20485007.68146574]
 [20794362.29739249]
 [21103716.91331923]
 [21413071.52924585]
 [21722426.1451726 ]
 [22031780.76109934]
 [22650489.9929527 ]
 [22959844.60887945]]


In [12]:
X_test.head()

mae_scores = []
r2_scores=[]

mae = mean_absolute_error(y_test, Y_pred)
r2 = r2_score(y_test, Y_pred)

mae_scores.append(mae)
r2_scores.append(r2)

print("MAE moyen :", np.mean(mae_scores))
print("R2 moyen :", np.mean(r2_scores))

MAE moyen : 2523494.5979490397
R2 moyen : -1.666626432409184


In [13]:
X_all = df[["annee"]].astype(int)
y_all = df["total"]

regressor_final = LinearRegression()
regressor_final.fit(X_all, y_all)

,"fit_intercept fit_intercept: bool, default=TrueWhether to calculate the intercept for this model. If setto False, no intercept will be used in calculations(i.e. data is expected to be centered).",True
,"copy_X copy_X: bool, default=TrueIf True, X will be copied; else, it may be overwritten.",True
,"tol tol: float, default=1e-6The precision of the solution (`coef_`) is determined by `tol` whichspecifies a different convergence criterion for the `lsqr` solver.`tol` is set as `atol` and `btol` of :func:`scipy.sparse.linalg.lsqr` whenfitting on sparse training data. This parameter has no effect when fittingon dense data... versionadded:: 1.7",1e-06
,"n_jobs n_jobs: int, default=NoneThe number of jobs to use for the computation. This will only providespeedup in case of sufficiently large problems, that is if firstly`n_targets > 1` and secondly `X` is sparse or if `positive` is setto `True`. ``None`` means 1 unless in a:obj:`joblib.parallel_backend` context. ``-1`` means using allprocessors. See :term:`Glossary ` for more details.",None
,"positive positive: bool, default=FalseWhen set to ``True``, forces the coefficients to be positive. Thisoption is only supported for dense arrays.For a comparison between a linear regression model with positive constraintson the regression coefficients and a linear regression without such constraints,see :ref:`sphx_glr_auto_examples_linear_model_plot_nnls.py`... versionadded:: 0.24",False


In [14]:
#Predictions 2021 w/t covid effect
year_2021 = pd.DataFrame({"annee": [2021]})
pred_2021 = regressor_final.predict(year_2021)

print(f"Pred 2021 : {pred_2021}")


Pred 2021 : [19264383.75715226]


In [15]:
#Predictions 2024
year_2024 = pd.DataFrame({"annee": [2024]})
pred_2024 = regressor_final.predict(year_2024)
print(f"Pred 2024 : {pred_2024}")


real_2024 = np.array([51390962])

mae_2024 = mean_absolute_error(real_2024, pred_2024)
print(f"MAE 2024 : {mae_2024}")


Pred 2024 : [20018762.13986701]
MAE 2024 : 31372199.860132992


In [16]:
future_years = pd.DataFrame({"annee": [2025,2026,2027,2028,2029,2030]})
future_pred = regressor_final.predict(future_years)

df_future_pred = pd.DataFrame({
    "annee": future_years["annee"].values,
    "total_pred": future_pred
})

print(df_future_pred)

   annee    total_pred
0   2025  2.027022e+07
1   2026  2.052168e+07
2   2027  2.077314e+07
3   2028  2.102460e+07
4   2029  2.127606e+07
5   2030  2.152752e+07
